# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_extraction.text import (
    CountVectorizer
)
from sklearn.feature_selection import (
    SelectKBest, SelectPercentile,
    SelectFpr, VarianceThreshold,
    SelectFromModel,
    chi2, f_classif,
)
from sklearn.ensemble import (
    RandomForestClassifier
)
from sklearn.linear_model import (
    LogisticRegression
)
from sklearn.pipeline import (
    FeatureUnion, Pipeline
)
from scipy.sparse import hstack

from sklearn.naive_bayes import BernoulliNB

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
display(
    X_train.isnull().sum()[X_train.isnull().sum() > 0],
    X_test.isnull().sum()[X_test.isnull().sum() > 0]
)

funder                3635
installer             3655
subvillage             371
public_meeting        3334
scheme_management     3877
scheme_name          28166
permit                3056
dtype: int64

funder                869
installer             877
subvillage             99
public_meeting        821
scheme_management     969
scheme_name          7092
permit                737
dtype: int64

In [5]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [6]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [7]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

X_train.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_train.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))
X_test.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_test.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))


X_train.loc[:, 'date_recorded_month'] /= 12
X_test.loc[:, 'date_recorded_month'] /= 12

In [8]:
X_test.loc[:,'date_recorded_year'].fillna(0, inplace=True)
X_train.loc[:,'date_recorded_year'].replace([0, np.nan], X_test.loc[:,'date_recorded_year'].median(), inplace=True)
X_test.loc[:,'date_recorded_year'].replace([0, np.nan], X_test.loc[:,'date_recorded_year'].median(), inplace=True)
X_test.loc[:,'construction_year'].fillna(0, inplace=True)
X_train.loc[:,'construction_year'].replace([0, np.nan], X_test.loc[:,'construction_year'].median(), inplace=True)
X_test.loc[:,'construction_year'].replace([0, np.nan], X_test.loc[:,'construction_year'].median(), inplace=True)

In [9]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [10]:
X_train.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)
X_test.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)

In [11]:
heightscale = StandardScaler().fit(pd.concat((X_train.loc[:, 'gps_height'],
                                              X_test.loc[:, 'gps_height']), ignore_index=True))
X_train.loc[:, 'gps_height'] = heightscale.transform(X_train.loc[:, 'gps_height']).reshape(-1, 1)
X_test.loc[:, 'gps_height'] = heightscale.transform(X_test.loc[:, 'gps_height']).reshape(-1, 1)

/home/god/py/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn

In [12]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population'])
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population'])
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_train.loc[:,'amount_per_man'] /= X_test.loc[:,'amount_per_man'].max()
X_test.loc[:,'amount_per_man'] /= X_test.loc[:,'amount_per_man'].max()
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)

X_train.loc[:,'population'] /= X_test.loc[:,'population'].max()
X_test.loc[:,'population'] /= X_test.loc[:,'population'].max()

# display(X_test.loc[:,'amount_per_man'])
#amntpmscaler = StandardScaler().fit(X_test.loc[:, 'amount_per_man'])
#X_train.loc[:,'amount_per_man'] = amntpmscaler.transform(X_train.loc[:,'amount_per_man']).reshape(-1, 1)
#X_test.loc[:,'amount_per_man'] = amntpmscaler.transform(X_test.loc[:,'amount_per_man']).reshape(-1, 1)

In [13]:
X_train.loc[:,'gps_height'].replace([np.inf, -np.inf, np.nan, 0],
                                    X_train.loc[:,'gps_height'].median(), inplace=True)
X_test.loc[:,'gps_height'].replace([np.inf, -np.inf, np.nan, 0],
                                    X_train.loc[:,'gps_height'].median(), inplace=True)

In [14]:
amounttshscaler = StandardScaler().fit(pd.concat((X_train.loc[:, 'amount_tsh'],
                                              X_test.loc[:, 'amount_tsh']), ignore_index=True))
X_train.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_train.loc[:, 'amount_tsh']).reshape(-1, 1)
X_test.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_test.loc[:, 'amount_tsh']).reshape(-1, 1)

/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature o

In [15]:
X_train.loc[:,'age'].fillna(X_test.loc[:,'age'].median(), inplace=True)
X_test.loc[:,'age'].fillna(X_test.loc[:,'age'].median(), inplace=True)

In [16]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [17]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [18]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [19]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [20]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [21]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [22]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [23]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [24]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [25]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [26]:
drop_cats = ['source_type', 'source_class',
             'water_quality', 'quality_group',
             'quantity_group', 'waterpoint_type_group',
             'payment_type', 'extraction_type_group',
             'extraction_type_class', 'management_group',
             'construction_year', 'date_recorded_month'
]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [27]:
X_train.iloc[0,:]

amount_tsh                                         1.95453
funder                                               Roman
gps_height                                         1.04558
installer                                            Roman
longitude                                          34.9381
latitude                                          -9.85632
wpt_name                                                 1
basin                                           Lake Nyasa
subvillage                                        Mnyusi B
region                                              Iringa
region_code                                             11
district_code                                            5
lga                                                 Ludewa
ward                                              Mundindi
population                                      0.00950388
public_meeting                                           1
scheme_management                                      V

In [28]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []
text_selfeat_names = []

In [29]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [30]:
# lonlat shall be clusterized before usage
# day probably doesn't matter, but month might be (seasonality)
# the years in test and train do not intersect
X_train.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')
X_test.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')

In [31]:
display(X_train.shape, X_test.shape)

(59400, 28)

(14850, 28)

In [32]:
sel = Pipeline([
    ('varthres', VarianceThreshold(1e-5)),
    ('chi2', SelectFpr(chi2, 1e-24)),
    #('interact', PolynomialFeatures(2, interaction_only=True)),
    #('feats', FeatureUnion([
        # ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        # ('chi2', SelectKBest(chi2, min(128, tr.shape[1])).fit(tr, ytr),)
        # OHE-encoded features are supposed to have additive impact, so linear model:
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=100, max_depth=30)))
    #])),
])
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = CountVectorizer(ngram_range=(1,2), binary=True)
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)

    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    for i in range(tr.shape[1]):
        text_selfeat_names.append('%s_%s' % (cat, i))
    #tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    #te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    if tr.shape[1] > 0:
        text_feats_train.append(tr)
        text_feats_test.append(te)

text_feats_train = hstack(text_feats_train) # pd.DataFrame(hstack(text_feats_train), columns=text_selfeat_names)
text_feats_test = hstack(text_feats_test) # pd.DataFrame(hstack(text_feats_test), columns=text_selfeat_names)
# print(sel.fit_transform(text_feats_train, ytr).toarray().shape)
# sel_text_feats = SelectKBest(chi2, 48).fit(text_feats_train, ytr) # SelectFpr(f_classif, 1e-19)
sel_text_feats = sel
text_feats_train = sel.fit_transform(text_feats_train, ytr)
text_feats_test = sel.transform(text_feats_test)
print('cat features shape: ', text_feats_train.shape)
# print('top catfeats: ', sel.named_steps['linear'].estimator_.coef_[:20] )
del tr, te

X_train = X_train.drop(text_features, axis=1, errors='ignore')
cat_features = np.array(list(range(X_train.shape[1], # + berntr.shape[1],
                          X_train.shape[1] + text_feats_train.shape[1]))) # + berntr.shape[1] ))
X_train = pd.concat([X_train] +
                    [pd.DataFrame(text_feats_train.toarray())],
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] +
                   [pd.DataFrame(text_feats_test.toarray())],
                    axis=1)

cat features shape:  (59400, 35)


In [33]:
X_train.columns

Index([    'amount_tsh',     'gps_height',       'wpt_name',     'population',
       'public_meeting',         'permit',     'dry_season',   'rainy_season',
                  'age', 'amount_per_man',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27,               28,               29,
                     30,               31,               32,               33,
                     34],
      dtype='object')

In [34]:
X_train.head()

,amount_tsh,gps_height,wpt_name,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,...,25,26,27,28,29,30,31,32,33,34
0,1.95452941861,1.04558069781,1,0.00950388002441,1.0,0.0,1,0,12,0.00183486238532,...,0,0,1,0,1,0,0,0,1,0
1,-0.109636705889,1.05857227813,1,0.02441363676,0.5,1.0,1,0,3,0.0,...,0,0,0,0,0,1,0,0,1,0
2,-0.101036013704,0.0293504149544,1,0.0217978899643,1.0,1.0,1,0,4,3.33333333333e-06,...,0,0,0,0,0,1,0,0,0,0
3,-0.109636705889,-0.581253860116,1,0.00505711047171,1.0,1.0,1,0,27,0.0,...,1,1,1,0,0,0,0,0,0,0
4,-0.109636705889,-0.960896707264,1,0.0,1.0,1.0,0,0,25,0.0,...,0,0,0,0,0,1,0,0,1,0


In [35]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('dry_season', dtype('O')),
 ('rainy_season', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [36]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)

In [37]:
bern = BernoulliNB()
bern.fit(xtr.iloc[:, cat_features], ytr)
logit = LogisticRegression().fit(xtr.iloc[:, cat_features], ytr)

xtr = pd.concat([xtr.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xtr.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xtr.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xcv = pd.concat([xcv.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xcv.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xcv.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xva = pd.concat([xva.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xva.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xva.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xte = pd.concat([xte.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xte.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xte.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

In [38]:
xva.head()

,amount_tsh,gps_height,wpt_name,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,...,31,32,33,34,p_0,p_1,p_2,li_0,li_1,li_2
0,-0.109636705889,-0.292552075213,1,0.00697532478856,0.5,1.0,1,0,0,0.0,...,0,0,1,0,0.317524,0.306990,0.375486,0.515535,0.175124,0.309341
1,-0.107572539765,0.881020680419,1,0.00217978899643,1.0,1.0,0,0,5,8e-06,...,0,0,1,0,0.916094,0.076062,0.007844,0.831734,0.075486,0.092780
2,-0.109636705889,1.04413718889,1,0.00174383119714,0.5,1.0,1,0,3,0.0,...,0,0,1,0,0.230629,0.146802,0.622569,0.315311,0.065143,0.619546
3,-0.109636705889,-0.960896707264,1,0.0,1.0,1.0,0,0,25,0.0,...,0,0,1,0,0.663283,0.244381,0.092336,0.653394,0.178845,0.167761
4,-0.109636705889,1.8279625349,1,0.0517917865551,1.0,1.0,1,0,7,0.0,...,1,1,0,0,0.781979,0.019904,0.198118,0.714192,0.033572,0.252236


In [39]:
display(xtr.shape, xte.shape)

(41578, 51)

(14850, 51)

In [40]:
#X_train.to_csv('X_train_prepr2.csv')
#X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
#pd.Series(ytr).to_csv('y_train_prepr2.csv')
#pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [41]:
import catboost

In [42]:
clf = catboost.CatBoostClassifier(2800, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importance=True)

In [43]:
#clf.fit(xtr, ytr, cat_features=cat_features,
#        eval_set=(xcv, ycv), verbose=True)

Borders generated
0:	learn 0.7160758093	test 0.7176695106	bestTest 0.7176695106	passed: 0.757 sec	total: 937ms	remaining: 43m 41s
1:	learn 0.7182163644	test 0.7192411316	bestTest 0.7192411316	passed: 0.838 sec	total: 1.77s	remaining: 41m 22s
2:	learn 0.7173745731	test 0.7189043556	bestTest 0.7192411316	passed: 0.753 sec	total: 2.53s	remaining: 39m 16s
3:	learn 0.7185049786	test 0.7199146834	bestTest 0.7199146834	passed: 0.765 sec	total: 3.29s	remaining: 38m 21s
4:	learn 0.717494829	test 0.7191288729	bestTest 0.7199146834	passed: 0.717 sec	total: 4.01s	remaining: 37m 21s
5:	learn 0.7174707778	test 0.7191288729	bestTest 0.7199146834	passed: 0.711 sec	total: 4.72s	remaining: 36m 38s
6:	learn 0.7175429314	test 0.7199146834	bestTest 0.7199146834	passed: 0.672 sec	total: 5.39s	remaining: 35m 52s
7:	learn 0.7178555967	test 0.7207004939	bestTest 0.7207004939	passed: 0.192 sec	total: 5.58s	remaining: 32m 29s
8:	learn 0.7179758526	test 0.7210372699	bestTest 0.7210372699	passed: 0.629 sec	total: 

73:	learn 0.7233873683	test 0.7250785811	bestTest 0.725415357	passed: 0.563 sec	total: 47.1s	remaining: 28m 55s
74:	learn 0.7233633171	test 0.7249663224	bestTest 0.725415357	passed: 0.732 sec	total: 47.8s	remaining: 28m 58s
75:	learn 0.7233152148	test 0.7249663224	bestTest 0.725415357	passed: 0.662 sec	total: 48.5s	remaining: 28m 58s
76:	learn 0.7230987541	test 0.7248540638	bestTest 0.725415357	passed: 0.27 sec	total: 48.8s	remaining: 28m 44s
77:	learn 0.7231949589	test 0.7247418051	bestTest 0.725415357	passed: 0.693 sec	total: 49.5s	remaining: 28m 46s
78:	learn 0.7232190101	test 0.7246295465	bestTest 0.725415357	passed: 0.554 sec	total: 50s	remaining: 28m 42s
79:	learn 0.7231468565	test 0.7246295465	bestTest 0.725415357	passed: 0.614 sec	total: 50.6s	remaining: 28m 41s
80:	learn 0.7232430612	test 0.7246295465	bestTest 0.725415357	passed: 0.639 sec	total: 51.3s	remaining: 28m 41s
81:	learn 0.7233152148	test 0.7249663224	bestTest 0.725415357	passed: 0.662 sec	total: 51.9s	remaining: 28m

146:	learn 0.7338977344	test 0.7364167041	bestTest 0.7364167041	passed: 0.921 sec	total: 1m 35s	remaining: 28m 50s
147:	learn 0.7340179903	test 0.73675348	bestTest 0.73675348	passed: 0.853 sec	total: 1m 36s	remaining: 28m 53s
148:	learn 0.7348116793	test 0.7370902559	bestTest 0.7370902559	passed: 0.635 sec	total: 1m 37s	remaining: 28m 52s
149:	learn 0.7349559863	test 0.7372025146	bestTest 0.7372025146	passed: 0.81 sec	total: 1m 38s	remaining: 28m 54s
150:	learn 0.7350521911	test 0.7372025146	bestTest 0.7372025146	passed: 0.686 sec	total: 1m 38s	remaining: 28m 54s
151:	learn 0.7350521911	test 0.7374270319	bestTest 0.7374270319	passed: 0.722 sec	total: 1m 39s	remaining: 28m 54s
152:	learn 0.7351483958	test 0.7375392905	bestTest 0.7375392905	passed: 0.382 sec	total: 1m 39s	remaining: 28m 49s
153:	learn 0.7354610611	test 0.7375392905	bestTest 0.7375392905	passed: 0.768 sec	total: 1m 40s	remaining: 28m 50s
154:	learn 0.7354129588	test 0.7373147732	bestTest 0.7375392905	passed: 0.931 sec	tot

218:	learn 0.7414017028	test 0.7416928604	bestTest 0.7421418949	passed: 0.511 sec	total: 2m 25s	remaining: 28m 37s
219:	learn 0.741425754	test 0.7415806017	bestTest 0.7421418949	passed: 0.757 sec	total: 2m 26s	remaining: 28m 38s
220:	learn 0.7415219587	test 0.7415806017	bestTest 0.7421418949	passed: 0.828 sec	total: 2m 27s	remaining: 28m 39s
221:	learn 0.7416181635	test 0.7416928604	bestTest 0.7421418949	passed: 1.06 sec	total: 2m 28s	remaining: 28m 43s
222:	learn 0.7415700611	test 0.7416928604	bestTest 0.7421418949	passed: 0.339 sec	total: 2m 28s	remaining: 28m 38s
223:	learn 0.7415941123	test 0.7416928604	bestTest 0.7421418949	passed: 0.366 sec	total: 2m 29s	remaining: 28m 34s
224:	learn 0.7416181635	test 0.7419173776	bestTest 0.7421418949	passed: 0.698 sec	total: 2m 29s	remaining: 28m 34s
225:	learn 0.7415700611	test 0.7415806017	bestTest 0.7421418949	passed: 0.905 sec	total: 2m 30s	remaining: 28m 36s
226:	learn 0.7415460099	test 0.7415806017	bestTest 0.7421418949	passed: 0.701 sec	

290:	learn 0.7456347107	test 0.7446115851	bestTest 0.7447238437	passed: 0.675 sec	total: 3m 14s	remaining: 27m 57s
291:	learn 0.7456587618	test 0.7447238437	bestTest 0.7447238437	passed: 0.462 sec	total: 3m 14s	remaining: 27m 54s
292:	learn 0.745682813	test 0.7447238437	bestTest 0.7447238437	passed: 0.331 sec	total: 3m 15s	remaining: 27m 51s
293:	learn 0.7458992737	test 0.7448361024	bestTest 0.7448361024	passed: 0.662 sec	total: 3m 15s	remaining: 27m 50s
294:	learn 0.7457309154	test 0.7448361024	bestTest 0.7448361024	passed: 0.802 sec	total: 3m 16s	remaining: 27m 51s
295:	learn 0.7458271201	test 0.7450606197	bestTest 0.7450606197	passed: 0.932 sec	total: 3m 17s	remaining: 27m 52s
296:	learn 0.7458271201	test 0.7450606197	bestTest 0.7450606197	passed: 0.462 sec	total: 3m 18s	remaining: 27m 50s
297:	learn 0.7459233248	test 0.7447238437	bestTest 0.7450606197	passed: 0.861 sec	total: 3m 19s	remaining: 27m 51s
298:	learn 0.7458511713	test 0.7447238437	bestTest 0.7450606197	passed: 1.04 sec	

...

2505:	learn 0.7877242773	test 0.7652671756	bestTest 0.7653794342	passed: 0.492 sec	total: 29m 22s	remaining: 3m 26s
2506:	learn 0.7877002261	test 0.7652671756	bestTest 0.7653794342	passed: 0.565 sec	total: 29m 23s	remaining: 3m 26s
2507:	learn 0.7877242773	test 0.7651549169	bestTest 0.7653794342	passed: 0.505 sec	total: 29m 23s	remaining: 3m 25s
2508:	learn 0.7877483284	test 0.7652671756	bestTest 0.7653794342	passed: 0.66 sec	total: 29m 24s	remaining: 3m 24s
2509:	learn 0.7877483284	test 0.7652671756	bestTest 0.7653794342	passed: 0.786 sec	total: 29m 25s	remaining: 3m 23s
2510:	learn 0.7877723796	test 0.7651549169	bestTest 0.7653794342	passed: 0.893 sec	total: 29m 26s	remaining: 3m 23s
2511:	learn 0.7877723796	test 0.7651549169	bestTest 0.7653794342	passed: 0.938 sec	total: 29m 27s	remaining: 3m 22s
2512:	learn 0.7877483284	test 0.7651549169	bestTest 0.7653794342	passed: 0.804 sec	total: 29m 28s	remaining: 3m 21s
2513:	learn 0.787820482	test 0.7651549169	bestTest 0.7653794342	passed: 0

2577:	learn 0.788349608	test 0.7653794342	bestTest 0.7656039515	passed: 0.899 sec	total: 30m 14s	remaining: 2m 36s
2578:	learn 0.7884458127	test 0.7653794342	bestTest 0.7656039515	passed: 0.883 sec	total: 30m 15s	remaining: 2m 35s
2579:	learn 0.7884217615	test 0.7653794342	bestTest 0.7656039515	passed: 0.619 sec	total: 30m 16s	remaining: 2m 34s
2580:	learn 0.7883255568	test 0.7654916929	bestTest 0.7656039515	passed: 0.713 sec	total: 30m 17s	remaining: 2m 34s
2581:	learn 0.7883015056	test 0.7654916929	bestTest 0.7656039515	passed: 0.762 sec	total: 30m 17s	remaining: 2m 33s
2582:	learn 0.7883736591	test 0.7656039515	bestTest 0.7656039515	passed: 0.812 sec	total: 30m 18s	remaining: 2m 32s
2583:	learn 0.7884217615	test 0.7654916929	bestTest 0.7656039515	passed: 0.789 sec	total: 30m 19s	remaining: 2m 32s
2584:	learn 0.7884458127	test 0.7654916929	bestTest 0.7656039515	passed: 0.732 sec	total: 30m 20s	remaining: 2m 31s
2585:	learn 0.7884458127	test 0.7654916929	bestTest 0.7656039515	passed: 

2649:	learn 0.7886863245	test 0.7652671756	bestTest 0.7656039515	passed: 0.826 sec	total: 31m 6s	remaining: 1m 45s
2650:	learn 0.788614171	test 0.7651549169	bestTest 0.7656039515	passed: 0.838 sec	total: 31m 7s	remaining: 1m 44s
2651:	learn 0.7886382221	test 0.7651549169	bestTest 0.7656039515	passed: 0.834 sec	total: 31m 8s	remaining: 1m 44s
2652:	learn 0.7886382221	test 0.7651549169	bestTest 0.7656039515	passed: 0.857 sec	total: 31m 9s	remaining: 1m 43s
2653:	learn 0.7886382221	test 0.7651549169	bestTest 0.7656039515	passed: 0.854 sec	total: 31m 10s	remaining: 1m 42s
2654:	learn 0.7887103757	test 0.7652671756	bestTest 0.7656039515	passed: 0.789 sec	total: 31m 11s	remaining: 1m 42s
2655:	learn 0.788758478	test 0.7653794342	bestTest 0.7656039515	passed: 0.696 sec	total: 31m 11s	remaining: 1m 41s
2656:	learn 0.788758478	test 0.7653794342	bestTest 0.7656039515	passed: 0.849 sec	total: 31m 12s	remaining: 1m 40s
2657:	learn 0.7888065804	test 0.7654916929	bestTest 0.7656039515	passed: 0.637 

2720:	learn 0.7893838087	test 0.7657162101	bestTest 0.7658284688	passed: 0.743 sec	total: 31m 58s	remaining: 55.7s
2721:	learn 0.7894078599	test 0.7657162101	bestTest 0.7658284688	passed: 0.801 sec	total: 31m 59s	remaining: 55s
2722:	learn 0.7895040646	test 0.7658284688	bestTest 0.7658284688	passed: 0.693 sec	total: 32m	remaining: 54.3s
2723:	learn 0.7895040646	test 0.7659407274	bestTest 0.7659407274	passed: 0.631 sec	total: 32m	remaining: 53.6s
2724:	learn 0.7895040646	test 0.7658284688	bestTest 0.7659407274	passed: 0.595 sec	total: 32m 1s	remaining: 52.9s
2725:	learn 0.7895040646	test 0.7660529861	bestTest 0.7660529861	passed: 0.786 sec	total: 32m 2s	remaining: 52.2s
2726:	learn 0.7895762182	test 0.7660529861	bestTest 0.7660529861	passed: 0.709 sec	total: 32m 2s	remaining: 51.5s
2727:	learn 0.7895281158	test 0.7658284688	bestTest 0.7660529861	passed: 0.816 sec	total: 32m 3s	remaining: 50.8s
2728:	learn 0.789552167	test 0.7658284688	bestTest 0.7660529861	passed: 0.68 sec	total: 32m 4s

2792:	learn 0.7899610371	test 0.7659407274	bestTest 0.7661652447	passed: 0.644 sec	total: 32m 50s	remaining: 4.94s
2793:	learn 0.7899369859	test 0.7659407274	bestTest 0.7661652447	passed: 0.708 sec	total: 32m 50s	remaining: 4.23s
2794:	learn 0.7899129347	test 0.7658284688	bestTest 0.7661652447	passed: 0.787 sec	total: 32m 51s	remaining: 3.53s
2795:	learn 0.7899850883	test 0.7658284688	bestTest 0.7661652447	passed: 0.777 sec	total: 32m 52s	remaining: 2.82s
2796:	learn 0.7899850883	test 0.7660529861	bestTest 0.7661652447	passed: 0.655 sec	total: 32m 53s	remaining: 2.12s
2797:	learn 0.7900331906	test 0.7660529861	bestTest 0.7661652447	passed: 0.914 sec	total: 32m 54s	remaining: 1.41s
2798:	learn 0.7901293954	test 0.7661652447	bestTest 0.7661652447	passed: 0.58 sec	total: 32m 54s	remaining: 706ms
2799:	learn 0.7900572418	test 0.7661652447	bestTest 0.7661652447	passed: 0.607 sec	total: 32m 55s	remaining: 0us

bestTest = 0.7661652447
bestIteration = 2799



In [44]:
#clf.score(xcv, ycv)

0.76616524472384373

In [45]:
#clf.score(xva, yva)

0.77215615885124522

In [46]:
y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)

ytr2 = y_bin.transform(ytr)
ycv2 = y_bin.transform(ycv)
yva2 = y_bin.transform(yva)

xscaler = StandardScaler().fit(xte)
xtr2 = xscaler.transform(xtr)
xte2 = xscaler.transform(xte)
xcv2 = xscaler.transform(xcv)
xva2 = xscaler.transform(xva)

def y_to_df(y, ids):
    y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df


In [47]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.utils

Using TensorFlow backend.


In [53]:
clf = Sequential([
    Dense(2*xtr.shape[1], input_shape=(xtr.shape[1],), activation='softmax'),
    Dropout(.1),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(3, activation='softmax'),
])
clf.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [54]:
clf.fit(xtr2, ytr2,
      batch_size=256,
      epochs=3000)


Epoch 1/3000
41578/41578 [==============================] - 0s - loss: 1.0171 - acc: 0.5431     
Epoch 2/3000
41578/41578 [==============================] - 0s - loss: 0.9198 - acc: 0.5431     
Epoch 3/3000
41578/41578 [==============================] - 0s - loss: 0.8631 - acc: 0.5541     
Epoch 4/3000
41578/41578 [==============================] - 0s - loss: 0.8181 - acc: 0.7024     
Epoch 5/3000
41578/41578 [==============================] - 0s - loss: 0.7785 - acc: 0.7150     
Epoch 6/3000
41578/41578 [==============================] - 0s - loss: 0.7470 - acc: 0.7231     
Epoch 7/3000
41578/41578 [==============================] - 0s - loss: 0.7230 - acc: 0.7348     
Epoch 8/3000
41578/41578 [==============================] - 0s - loss: 0.7056 - acc: 0.7372     
Epoch 9/3000
41578/41578 [==============================] - 0s - loss: 0.6943 - acc: 0.7399     
Epoch 10/3000
41578/41578 [==============================] - 0s - loss: 0.6850 - acc: 0.7410     
Epoch 11/3000
41578/41578 [==

41578/41578 [==============================] - 0s - loss: 0.5678 - acc: 0.7686     
Epoch 85/3000
41578/41578 [==============================] - 0s - loss: 0.5660 - acc: 0.7700     
Epoch 86/3000
41578/41578 [==============================] - 0s - loss: 0.5658 - acc: 0.7708     
Epoch 87/3000
41578/41578 [==============================] - 0s - loss: 0.5653 - acc: 0.7695     
Epoch 88/3000
41578/41578 [==============================] - 0s - loss: 0.5644 - acc: 0.7700     
Epoch 89/3000
41578/41578 [==============================] - 0s - loss: 0.5638 - acc: 0.7697     
Epoch 90/3000
41578/41578 [==============================] - 0s - loss: 0.5634 - acc: 0.7715     
Epoch 91/3000
41578/41578 [==============================] - 0s - loss: 0.5655 - acc: 0.7694     
Epoch 92/3000
41578/41578 [==============================] - 0s - loss: 0.5634 - acc: 0.7716     
Epoch 93/3000
41578/41578 [==============================] - 0s - loss: 0.5636 - acc: 0.7707     
Epoch 94/3000
41578/41578 [=======

41578/41578 [==============================] - 0s - loss: 0.5456 - acc: 0.7757     
Epoch 168/3000
41578/41578 [==============================] - 0s - loss: 0.5442 - acc: 0.7767     
Epoch 169/3000
41578/41578 [==============================] - 0s - loss: 0.5445 - acc: 0.7771     
Epoch 170/3000
41578/41578 [==============================] - 0s - loss: 0.5453 - acc: 0.7771     
Epoch 171/3000
41578/41578 [==============================] - 0s - loss: 0.5455 - acc: 0.7763     
Epoch 172/3000
41578/41578 [==============================] - 0s - loss: 0.5469 - acc: 0.7761     
Epoch 173/3000
41578/41578 [==============================] - 0s - loss: 0.5465 - acc: 0.7770     
Epoch 174/3000
41578/41578 [==============================] - 0s - loss: 0.5447 - acc: 0.7769     
Epoch 175/3000
41578/41578 [==============================] - 0s - loss: 0.5449 - acc: 0.7773     
Epoch 176/3000
41578/41578 [==============================] - 0s - loss: 0.5441 - acc: 0.7781     
Epoch 177/3000
41578/4157

41578/41578 [==============================] - 0s - loss: 0.5359 - acc: 0.7805     
Epoch 250/3000
41578/41578 [==============================] - 0s - loss: 0.5363 - acc: 0.7800     
Epoch 251/3000
41578/41578 [==============================] - 0s - loss: 0.5361 - acc: 0.7790     
Epoch 252/3000
41578/41578 [==============================] - 0s - loss: 0.5357 - acc: 0.7811     
Epoch 253/3000
41578/41578 [==============================] - 0s - loss: 0.5360 - acc: 0.7784     
Epoch 254/3000
41578/41578 [==============================] - 0s - loss: 0.5343 - acc: 0.7811     
Epoch 255/3000
41578/41578 [==============================] - 0s - loss: 0.5367 - acc: 0.7797     
Epoch 256/3000
41578/41578 [==============================] - 0s - loss: 0.5361 - acc: 0.7811     
Epoch 257/3000
41578/41578 [==============================] - 0s - loss: 0.5339 - acc: 0.7803     
Epoch 258/3000
41578/41578 [==============================] - 0s - loss: 0.5352 - acc: 0.7784     
Epoch 259/3000
41578/4157

41578/41578 [==============================] - 0s - loss: 0.5294 - acc: 0.7819     
Epoch 332/3000
41578/41578 [==============================] - 0s - loss: 0.5278 - acc: 0.7842     
Epoch 333/3000
41578/41578 [==============================] - 0s - loss: 0.5288 - acc: 0.7827     
Epoch 334/3000
41578/41578 [==============================] - 0s - loss: 0.5281 - acc: 0.7822     
Epoch 335/3000
41578/41578 [==============================] - 0s - loss: 0.5280 - acc: 0.7837     
Epoch 336/3000
41578/41578 [==============================] - 0s - loss: 0.5285 - acc: 0.7827     
Epoch 337/3000
41578/41578 [==============================] - 0s - loss: 0.5282 - acc: 0.7815     
Epoch 338/3000
41578/41578 [==============================] - 0s - loss: 0.5292 - acc: 0.7827     
Epoch 339/3000
41578/41578 [==============================] - 0s - loss: 0.5272 - acc: 0.7831     
Epoch 340/3000
41578/41578 [==============================] - 0s - loss: 0.5278 - acc: 0.7825     
Epoch 341/3000
41578/4157

...

41578/41578 [==============================] - 0s - loss: 0.4762 - acc: 0.8027     
Epoch 2627/3000
41578/41578 [==============================] - 0s - loss: 0.4758 - acc: 0.8044     
Epoch 2628/3000
41578/41578 [==============================] - 0s - loss: 0.4767 - acc: 0.8022     
Epoch 2629/3000
41578/41578 [==============================] - 0s - loss: 0.4764 - acc: 0.8030     
Epoch 2630/3000
41578/41578 [==============================] - 0s - loss: 0.4732 - acc: 0.8035     
Epoch 2631/3000
41578/41578 [==============================] - 0s - loss: 0.4754 - acc: 0.8029     
Epoch 2632/3000
41578/41578 [==============================] - 0s - loss: 0.4746 - acc: 0.8025     
Epoch 2633/3000
41578/41578 [==============================] - 0s - loss: 0.4772 - acc: 0.8029     
Epoch 2634/3000
41578/41578 [==============================] - 0s - loss: 0.4727 - acc: 0.8043     
Epoch 2635/3000
41578/41578 [==============================] - 0s - loss: 0.4752 - acc: 0.8036     
Epoch 2636/3000


41578/41578 [==============================] - 0s - loss: 0.4754 - acc: 0.8032     
Epoch 2709/3000
41578/41578 [==============================] - 0s - loss: 0.4745 - acc: 0.8043     
Epoch 2710/3000
41578/41578 [==============================] - 0s - loss: 0.4741 - acc: 0.8026     
Epoch 2711/3000
41578/41578 [==============================] - 0s - loss: 0.4770 - acc: 0.8017     
Epoch 2712/3000
41578/41578 [==============================] - 0s - loss: 0.4755 - acc: 0.8029     
Epoch 2713/3000
41578/41578 [==============================] - 0s - loss: 0.4735 - acc: 0.8048     
Epoch 2714/3000
41578/41578 [==============================] - 0s - loss: 0.4748 - acc: 0.8042     
Epoch 2715/3000
41578/41578 [==============================] - 0s - loss: 0.4754 - acc: 0.8030     
Epoch 2716/3000
41578/41578 [==============================] - 0s - loss: 0.4727 - acc: 0.8055     
Epoch 2717/3000
41578/41578 [==============================] - 0s - loss: 0.4753 - acc: 0.8043     
Epoch 2718/3000


41578/41578 [==============================] - 0s - loss: 0.4747 - acc: 0.8030     
Epoch 2791/3000
41578/41578 [==============================] - 0s - loss: 0.4748 - acc: 0.8024     
Epoch 2792/3000
41578/41578 [==============================] - 0s - loss: 0.4726 - acc: 0.8043     
Epoch 2793/3000
41578/41578 [==============================] - 0s - loss: 0.4741 - acc: 0.8051     
Epoch 2794/3000
41578/41578 [==============================] - 0s - loss: 0.4755 - acc: 0.8034     
Epoch 2795/3000
41578/41578 [==============================] - 0s - loss: 0.4743 - acc: 0.8035     
Epoch 2796/3000
41578/41578 [==============================] - 0s - loss: 0.4743 - acc: 0.8040     
Epoch 2797/3000
41578/41578 [==============================] - 0s - loss: 0.4749 - acc: 0.8041     
Epoch 2798/3000
41578/41578 [==============================] - 0s - loss: 0.4752 - acc: 0.8024     
Epoch 2799/3000
41578/41578 [==============================] - 0s - loss: 0.4753 - acc: 0.8030     
Epoch 2800/3000


41578/41578 [==============================] - 0s - loss: 0.4745 - acc: 0.8048     
Epoch 2873/3000
41578/41578 [==============================] - 0s - loss: 0.4729 - acc: 0.8055     
Epoch 2874/3000
41578/41578 [==============================] - 0s - loss: 0.4742 - acc: 0.8049     
Epoch 2875/3000
41578/41578 [==============================] - 0s - loss: 0.4769 - acc: 0.8031     
Epoch 2876/3000
41578/41578 [==============================] - 0s - loss: 0.4758 - acc: 0.8015     
Epoch 2877/3000
41578/41578 [==============================] - 0s - loss: 0.4748 - acc: 0.8042     
Epoch 2878/3000
41578/41578 [==============================] - 0s - loss: 0.4746 - acc: 0.8035     
Epoch 2879/3000
41578/41578 [==============================] - 0s - loss: 0.4745 - acc: 0.8037     
Epoch 2880/3000
41578/41578 [==============================] - 0s - loss: 0.4739 - acc: 0.8048     
Epoch 2881/3000
41578/41578 [==============================] - 0s - loss: 0.4743 - acc: 0.8053     
Epoch 2882/3000


41578/41578 [==============================] - 0s - loss: 0.4754 - acc: 0.8036     
Epoch 2955/3000
41578/41578 [==============================] - 0s - loss: 0.4736 - acc: 0.8044     
Epoch 2956/3000
41578/41578 [==============================] - 0s - loss: 0.4747 - acc: 0.8033     
Epoch 2957/3000
41578/41578 [==============================] - 0s - loss: 0.4737 - acc: 0.8042     
Epoch 2958/3000
41578/41578 [==============================] - 0s - loss: 0.4738 - acc: 0.8048     
Epoch 2959/3000
41578/41578 [==============================] - 0s - loss: 0.4735 - acc: 0.8036     
Epoch 2960/3000
41578/41578 [==============================] - 0s - loss: 0.4740 - acc: 0.8041     
Epoch 2961/3000
41578/41578 [==============================] - 0s - loss: 0.4729 - acc: 0.8034     
Epoch 2962/3000
41578/41578 [==============================] - 0s - loss: 0.4733 - acc: 0.8031     
Epoch 2963/3000
41578/41578 [==============================] - 0s - loss: 0.4740 - acc: 0.8049     
Epoch 2964/3000


In [55]:
yva_pred2 = clf.predict_classes(xva2)
sklearn.metrics.accuracy_score(y_bin.inverse_transform(yva2), yva_pred2)

6272/8914 [====================>.........] - ETA: 0s

0.7447834866502131